In [1]:
data1 = ['这个', '程序', '代码', '太乱', '那个', '代码', '规范']
data2 = ['这个', '程序', '代码', '不', '规范', '那个', '更', '规范']
data3 = ['这个', '程序', '代码', '不', '规范', '那个', '规范', '些']

In [47]:
from datasketch import MinHash, MinHashLSH, MinHashLSHEnsemble, MinHashLSHForest
from simhash import Simhash, SimhashIndex

In [3]:
# 创建MinHash对象
m1 = MinHash()
m2 = MinHash()
m3 = MinHash()

In [4]:
for d in data1:
    m1.update(d.encode('utf8'))
for d in data2:
    m2.update(d.encode('utf8'))
for d in data3:
    m3.update(d.encode('utf8'))

In [5]:
m1.hashfunc, m1.count, m1.jaccard

(<function datasketch.hashfunc.sha1_hash32(data)>,
 <bound method MinHash.count of <datasketch.minhash.MinHash object at 0x00000246D4EBA888>>,
 <bound method MinHash.jaccard of <datasketch.minhash.MinHash object at 0x00000246D4EBA888>>)

### MinHash

In [6]:
print("MinHash预估的Jaccard相似度", m1.jaccard(m2))

MinHash预估的Jaccard相似度 0.6015625


In [7]:
s1 = set(data1)
s2 = set(data2)
actual_jaccard = float(len(s1.intersection(s2)) / float(len(s1.union(s2))))
print('Jaccard相似度实际值', actual_jaccard)
print(s1.intersection(s2))
print(len(s1.intersection(s2)))
print(len(s1.union(s2)))
print(len(s1.union(s2)))

Jaccard相似度实际值 0.625
{'程序', '规范', '这个', '代码', '那个'}
5
8
8


### MinHashLSH

In [20]:
lsh = MinHashLSH(threshold=0.5, num_perm=128)
lsh.insert("m2", m2)
lsh.insert("m3", m3)
result = lsh.query(m1)
print('近似邻居（Jaccard相似度>0.5）', result)

近似邻居（Jaccard相似度>0.5） ['m2', 'm3']


### MinHashLSHEnsemble

In [8]:
m1.hashvalues.shape

(128,)

In [9]:
# 创建LSH Ensemble
lshensemble = MinHashLSHEnsemble(threshold=0.8, num_perm=128)

In [10]:
# Index takes an iterable of (key, minhash, size)
lshensemble.index([("m2", m2, len(data2)), ("m3", m3, len(data3))])

In [11]:
lshensemble.h, lshensemble.xqs, lshensemble.threshold, lshensemble.params

(128, array([6.73794700e-03, 2.04680757e-02, 6.21765240e-02, 1.88875603e-01,
        5.73753421e-01, 1.74290900e+00, 5.29449005e+00, 1.60832407e+01,
        4.88565713e+01, 1.48413159e+02]), 0.8, array([[ 1,  8],
        [ 1,  8],
        [ 1,  8],
        [ 1,  8],
        [ 1,  8],
        [25,  4],
        [42,  3],
        [42,  3],
        [42,  3],
        [42,  3]]))

In [12]:
lshensemble.indexes

[{8: <datasketch.lsh.MinHashLSH at 0x246d4ed1188>,
  3: <datasketch.lsh.MinHashLSH at 0x246d4ed1808>,
  4: <datasketch.lsh.MinHashLSH at 0x246d4ed1f48>},
 {8: <datasketch.lsh.MinHashLSH at 0x246d4ed3a88>,
  3: <datasketch.lsh.MinHashLSH at 0x246d4ede348>,
  4: <datasketch.lsh.MinHashLSH at 0x246d4ecfac8>},
 {8: <datasketch.lsh.MinHashLSH at 0x246d33667c8>,
  3: <datasketch.lsh.MinHashLSH at 0x246d4ec0548>,
  4: <datasketch.lsh.MinHashLSH at 0x246d4ef4048>},
 {8: <datasketch.lsh.MinHashLSH at 0x246d4ef8108>,
  3: <datasketch.lsh.MinHashLSH at 0x246d4ef8988>,
  4: <datasketch.lsh.MinHashLSH at 0x246d4efef48>},
 {8: <datasketch.lsh.MinHashLSH at 0x246d4f06048>,
  3: <datasketch.lsh.MinHashLSH at 0x246d4f068c8>,
  4: <datasketch.lsh.MinHashLSH at 0x246d4f0ce88>},
 {8: <datasketch.lsh.MinHashLSH at 0x246d4f0ffc8>,
  3: <datasketch.lsh.MinHashLSH at 0x246d4f15848>,
  4: <datasketch.lsh.MinHashLSH at 0x246d4f1ae08>},
 {8: <datasketch.lsh.MinHashLSH at 0x246d4f1cec8>,
  3: <datasketch.lsh.MinH

In [13]:
# 判断lshensemble是否存在m2, m3
print("m2" in lshensemble)
print("m3" in lshensemble)
print("m1" in lshensemble)

True
True
False


In [14]:
print('与m1相似度大于0.8的集合:')
for key in lshensemble.query(m1, len(data1)):
    print(key)

与m1相似度大于0.8的集合:
m2
m3


### MinHashLSHForest

In [15]:
forest = MinHashLSHForest()

In [16]:
forest.add("m2", m2)
forest.add("m3", m3)

In [17]:
# 在检索前，需要使用index
forest.index()

In [18]:
print('m1' in forest)
print('m2' in forest)
print('m3' in forest)

False
True
True


In [19]:
result = forest.query(m1, 2)
print('Top 2:', result)

Top 2: ['m2', 'm3']


### MinHashLSHForest + news

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
import jieba.posseg as pseg
import re

In [22]:
with open('dataset/sentences.txt', encoding='UTF-8') as file:
    text = file.read()
sentences = re.split('[。！？]', text.replace('\n', ''))

In [23]:
sentences

['11月10日深夜到11日凌晨，杭州阿里巴巴园区光柱擎天、灯火辉煌、不眠不休',
 '这是第11年双11，一个稀疏平常的日子凭空成为全民购物狂欢本就不可思议，而更不可思议的是不断刷新的消费数据',
 '仅用时96秒，2019天猫双11总成交额便超过100亿元，这个速度，比2018年缩短了29秒',
 '随后，500亿元、1000亿元，每一个成交额小目标都以更快的速度实现',
 '截至中证君发稿，2019天猫双11前三小时总成交额达到1325.7亿元，已经超过2018年天猫双11全天2135亿元总成交额的62%',
 '2018年，阿里巴巴用15小时49分钟39秒时间，打破了2017年1682亿元的双11全天成交纪录，最终以27%的增速定格在2135亿元',
 '今年双11，阿里巴巴又将以怎样的增速、多长时间来打破自己的纪录，它能否一举突破3000亿元大关，同时提振全国内需',
 '作为双11购物狂欢盛典的鼻祖，阿里巴巴在过去三年双11全网成交额中占据三分之二天下，京东、苏宁以及近期势头凶猛的拼多多均处于“后来”但尚未“居上”状态',
 '首先来看阿里巴巴（天猫）战报',
 '据介绍，今年，超过1000万款商品登陆天猫双11，超100万款新品在天猫双11首发，来自78个国家和地区的跨境电商进口商品、全国1000个数字农业基地的高品质农产品、2000个传统制造业产业带的工厂直供好货，以新供给满足消费者的多元化需求',
 '00:01:36，2019天猫双11总成交额超100亿元',
 '2018年用时2分05秒，今年缩短29秒',
 '00:12:49，2019天猫双11总成交额超500亿元',
 '2018年用时26分3秒，今年缩短一半',
 '01:03:59，2019天猫双11总成交额超1000亿元',
 '比2018年快了43分钟27秒，比2017年快了将近8小时',
 '']

In [24]:
sentences = sentences[:-1]

In [25]:
sentences

['11月10日深夜到11日凌晨，杭州阿里巴巴园区光柱擎天、灯火辉煌、不眠不休',
 '这是第11年双11，一个稀疏平常的日子凭空成为全民购物狂欢本就不可思议，而更不可思议的是不断刷新的消费数据',
 '仅用时96秒，2019天猫双11总成交额便超过100亿元，这个速度，比2018年缩短了29秒',
 '随后，500亿元、1000亿元，每一个成交额小目标都以更快的速度实现',
 '截至中证君发稿，2019天猫双11前三小时总成交额达到1325.7亿元，已经超过2018年天猫双11全天2135亿元总成交额的62%',
 '2018年，阿里巴巴用15小时49分钟39秒时间，打破了2017年1682亿元的双11全天成交纪录，最终以27%的增速定格在2135亿元',
 '今年双11，阿里巴巴又将以怎样的增速、多长时间来打破自己的纪录，它能否一举突破3000亿元大关，同时提振全国内需',
 '作为双11购物狂欢盛典的鼻祖，阿里巴巴在过去三年双11全网成交额中占据三分之二天下，京东、苏宁以及近期势头凶猛的拼多多均处于“后来”但尚未“居上”状态',
 '首先来看阿里巴巴（天猫）战报',
 '据介绍，今年，超过1000万款商品登陆天猫双11，超100万款新品在天猫双11首发，来自78个国家和地区的跨境电商进口商品、全国1000个数字农业基地的高品质农产品、2000个传统制造业产业带的工厂直供好货，以新供给满足消费者的多元化需求',
 '00:01:36，2019天猫双11总成交额超100亿元',
 '2018年用时2分05秒，今年缩短29秒',
 '00:12:49，2019天猫双11总成交额超500亿元',
 '2018年用时26分3秒，今年缩短一半',
 '01:03:59，2019天猫双11总成交额超1000亿元',
 '比2018年快了43分钟27秒，比2017年快了将近8小时']

In [32]:
stop = [line.strip() for line in open('dataset/stopwords/baidu_stopwords.txt', encoding='UTF-8').readlines()]

In [34]:
len(stop)

1396

In [111]:
# item_text进行切词
def get_item_str(item_text):
    item_str = ""
    item = (pseg.cut(item_text))
    for i in list(item):
        # 去掉停用词
        if i.word not in list(stop) and i.word not in ['', ' ', '#', '，','：','《','》', '…']:
            item_str += i.word
            # tfidf_vectorized.fit_transform的输入需要空格分隔的单词
            item_str += ' '
    return item_str

In [112]:
# 对item_str创建MinHash
def get_minhash(item_str):
    tmp = MinHash()
    for d in item_str:
        tmp.update(d.encode('utf-8'))
    return tmp

In [113]:
# 得到分词后的documents
documents = []
for item_text in sentences:
    item_str = get_item_str(item_text)
    documents.append(item_str)

In [114]:
documents

['11 月 10 日 深夜 11 日 凌晨 杭州 阿里巴巴 园区 光柱 擎天 、 灯火辉煌 、 不眠不休 ',
 '11 年 双 11 一个 稀疏 平常 日子 凭空 全民 购物 狂欢 不可思议 更 不可思议 刷新 消费 数据 ',
 '仅 时 96 秒 2019 天猫 双 11 总 成交额 便 超过 100 亿元 速度 2018 年 缩短 29 秒 ',
 '随后 500 亿元 、 1000 亿元 一个 成交额 小 目标 都 更快 速度 ',
 '截至 中证君 发稿 2019 天猫 双 11 前 三 小时 总 成交额 1325.7 亿元 超过 2018 年 天猫 双 11 全天 2135 亿元 总 成交额 62 % ',
 '2018 年 阿里巴巴 15 小时 49 分钟 39 秒 时间 打破 2017 年 1682 亿元 双 11 全天 成交 纪录 最终 27 % 增速 定格 2135 亿元 ',
 '双 11 阿里巴巴 增速 、 多长时间 打破 纪录 一举 突破 3000 亿元 大关 提振 全国 内需 ',
 '双 11 购物 狂欢 盛典 鼻祖 阿里巴巴 三年 双 11 全网 成交额 中 占据 三分之二 天下 京东 、 苏宁 近期 势头 凶猛 拼 多多 均 处于 尚未 居 上 状态 ',
 '来看 阿里巴巴 （ 天猫 ） 战报 ',
 '据介绍 超过 1000 万款 商品 登陆 天猫 双 11 超 100 万款 新品 天猫 双 11 首发 来自 78 国家 地区 跨境 电商 进口商品 、 全国 1000 数字 农业 基地 高品质 农产品 、 2000 传统 制造业 产业带 工厂 直供 好 货 新 供给 消费者 多元化 需求 ',
 '00 : 01 : 36 2019 天猫 双 11 总 成交额 超 100 亿元 ',
 '2018 年 时 2 分 05 秒 缩短 29 秒 ',
 '00 : 12 : 49 2019 天猫 双 11 总 成交额 超 500 亿元 ',
 '2018 年 时 26 分 3 秒 缩短 一半 ',
 '01 : 03 : 59 2019 天猫 双 11 总 成交额 超 1000 亿元 ',
 '2018 年 快 43 分钟 27 秒 2017 年 快 将近 8 小时 ']

In [115]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    tmp = get_minhash(documents[i])
    minhash_list.append(tmp)
    forest.add(i, tmp)
# index所有key, 以便可以进行检索
forest.index()

In [116]:
minhash_list[0].count()

35.84421293528233

In [117]:
query = '2019天猫双11总成交额超过100亿元'

In [118]:
item_str = get_item_str(query)

In [119]:
minhash_query = get_minhash(item_str)

In [120]:
# 查询forest中与m1相似的top-k个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print('Top 3', result)

10 0.78125 00:01:362019天猫双11总成交额超100亿元
12 0.828125 00:12:492019天猫双11总成交额超500亿元
14 0.78125 01:03:592019天猫双11总成交额超1000亿元
Top 3 [10, 12, 14]


### SimHash

In [121]:
data = [
    '这个程序代码太乱,那个代码规范',
    '这个程序代码不规范,那个更规范',
    '我是佩奇，这是我的弟弟乔治'
]

data = [
    '这个 程序 代码 太乱 那个 代码 规范',
    '这个 程序 代码 不 规范 那个 更 规范',
    '我 是 佩奇 这 是 我的 弟弟 乔治'
]

In [122]:
vec = TfidfVectorizer()
D = vec.fit_transform(data)
voc = dict((i, w) for w, i in vec.vocabulary_.items())

In [123]:
print(D.getrow(0))

  (0, 7)	0.32060325720533567
  (0, 9)	0.32060325720533567
  (0, 3)	0.4215547553457735
  (0, 1)	0.6412065144106713
  (0, 6)	0.32060325720533567
  (0, 8)	0.32060325720533567


In [124]:
D.getrow(0).indices

array([7, 9, 3, 1, 6, 8])

In [125]:
voc

{8: '这个',
 6: '程序',
 1: '代码',
 3: '太乱',
 9: '那个',
 7: '规范',
 2: '佩奇',
 5: '我的',
 4: '弟弟',
 0: '乔治'}

In [126]:
# 生成Simhash
sh_list = []
for i in range(D.shape[0]):
    Di = D.getrow(i)
    # features表示 (token, weight)元祖形式的列表
    features = zip([voc[j] for j in Di.indices], Di.data)
    print(Di, '\n', features)
    sh_list.append(Simhash(features))

  (0, 7)	0.32060325720533567
  (0, 9)	0.32060325720533567
  (0, 3)	0.4215547553457735
  (0, 1)	0.6412065144106713
  (0, 6)	0.32060325720533567
  (0, 8)	0.32060325720533567 
  (0, 7)	0.7071067811865476
  (0, 9)	0.3535533905932738
  (0, 1)	0.3535533905932738
  (0, 6)	0.3535533905932738
  (0, 8)	0.3535533905932738 
  (0, 0)	0.5
  (0, 4)	0.5
  (0, 5)	0.5
  (0, 2)	0.5 


In [127]:
print(sh_list[0])

In [128]:
print(sh_list[0].distance(sh_list[1]))
print(sh_list[0].distance(sh_list[2]))
print(sh_list[1].distance(sh_list[2]))

16
32
38


### weibo.txt 测试

In [129]:
with open('dataset/weibos.txt', encoding='UTF-8') as file:
    text = file.read()
weibo = re.split('[。！？]', text.replace('\n', '').replace('\u200b', ''))

In [130]:
# 得到分词后的documents
weibo_documents = []
for item_text in weibo:
    item_str = get_item_str(item_text)
    weibo_documents.append(item_str)

In [131]:
weibo_documents

['斯科拉里 执教 国足 上 一届 里 皮 从头 芾 尾 很大 机会 入 世界杯 一届 没 几个 能用 归化 都 没用 ',
 '国 足 输给 叙利亚 里 皮 辞职 ',
 '新 主帅 球迷 关注 焦点 ',
 '舆论 倾向 三个 人 山东鲁能 主帅 李霄鹏 、 武汉 卓尔 主帅 李铁 、 前 广州 恒大 主帅 斯科拉里 ',
 '中国足协 态度 里 皮 请辞 去 意 已 决 ',
 '',
 '比赛 当晚 太太 西蒙内塔 女士 儿子 小里皮 都 现场 看 台上 观战 ',
 '辞职 后 里皮 改变 原有 计划 — — 赛后 第二天 会 迪拜 飞 回 意大利 ',
 '意味着 本来 没 打算 球队 管理层 中国足协 高层 赛后 第一 时间 内 辞职 对话 ',
 '辞职 善后工作 包括 合同 沟通 工作 要待 日后 进一步 协商 ',
 '回顾 国 足 历届 外籍 教练 — — 里 皮 佩兰 卡马乔 杜伊 科维奇 阿里 · 汉 米卢 ',
 '一个 一个 有名 一个 一个 水 国足 踢 不好 足协 足协 不 解散 重组 天王老子 请来 都 不行 斯科拉里 想 执教 中国 国 足 ',
 '老头 凡是 里 皮 干 地方 想 试试 ',
 '老头 世界杯 冠军 教头 折 中国 没 丢人 毕竟 里 皮 折 ',
 '试试 ',
 '斯科拉里 水平 还 里 皮 ',
 '斯科拉里 看好 国 足 年薪 辞职 ',
 '中国 足球 不 名帅 不 外籍 教练 一点儿 毛用 ',
 '施拉普纳 二十余年 间 中国 足球 竟然 大踏步 倒退 一点儿 杀 不住 车 奶奶 刹车 系统 坏 ',
 '穿着 几百块 钱 球衣 几千块 钱 球鞋 几万块 钱 包 几十万 包机 几百万 上 千万 年薪 赛后 叙利亚 主教练 更衣室 里 队员 一个 耳光 ',
 '主教练 说 赛前 老子 再三 交代 一场 无论如何 都 赢 中国队 ',
 '中国 援助 粮食 美金 不再 援助 国家 狗日 些 吃 土 去 ',
 '球员 委屈 说 七十多 分钟 晓得 龟儿子 家 球门 踢 ',
 '里 皮 辞职 返回 意大利 助教 马达 洛尼 随队 返回 广州 ',
 '马达 洛尼 接受 采访 时 还原 更衣室 中 情况 更衣室 球员 都 试图 说服 里 皮 收回 队长 郑智 尝试 阻止 足协 代表 希望 考虑一下 

In [132]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(weibo_documents)):
    tmp = get_minhash(weibo_documents[i])
    minhash_list.append(tmp)
    forest.add(i, tmp)
# index所有key, 以便可以进行检索
forest.index()

In [133]:
query = '2022年世界杯冠军是谁？'

In [134]:
minhash_query = get_minhash(get_item_str(query))

In [137]:
# 查询forest中与m1相似的top-k个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), weibo_documents[result[i]].replace(' ', ''))
print('Top 3', result)

16 0.0546875 斯科拉里看好国足年薪辞职
13 0.2265625 老头世界杯冠军教头折中国没丢人毕竟里皮折
29 0.328125 带队绝不会比里皮更差能带国足夺得2022世界杯冠军
Top 3 [16, 13, 29]


In [138]:
result

[16, 13, 29]